# Task: Image compression
Analyze the three images present on ILIAS to decide which compression strategy is
the most promising one. Below, we suggest two such strategies, namely 2D run-length
encoding and quadtree representation. Choose one of them, or yet another strategy of
your own design. Regardless of the compression method you choose, write an algorithm
to decompress the image back to the original image.

In [1]:
#Imports
from __future__ import annotations

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from cv2 import Mat
from enum import Enum

class QuadTree:
    image: Mat
    root_partitions: list[QuadTreeNode]
    root_partition_size: tuple[int, int]

    class ControlByteSize(Enum):
        DIMENSION = 4
        PARTITION_SIZE = 2

    def __init__(self, image: Mat) -> None:
        self.image = image
        self.root_partitions = []

    def build_quad(self, roots_shape: tuple):
        self.root_partition_size = roots_shape
        image_width, image_height, _ = self.image.shape
        root_w, root_h = roots_shape

        print(f"Processing image {image_width}*{image_height}")

        if image_width % root_w != 0:
            raise Exception(f"Can not partition image with width {image_width} by partition of width {root_w}")
        if image_height % root_h != 0:
            raise Exception(f"Can not partition image with height {image_height} by partition of height {root_h}")
        
        if root_w != root_h:
            raise Exception(f"Can not partition image with partition shape {roots_shape} as it is not square")
        
        if (root_w % 2 != 0):
            raise Exception(f"Can not partition image with shape {roots_shape} as its dimensions are not divisible by 2")
        
        #Start the quad tree's roots, each root building its children recursively
        for x in range(0, image_width, root_w):
            for y in range(0, image_height, root_h):
                x1,x2 = x, x+root_w
                y1,y2 = y, y+root_h
                image_partition = self.image[x1:x2, y1:y2]
                self.root_partitions.append(QuadTreeNode(image_partition, (x1, x2, y1, y2)))
        
        total_encoded = sum([root.count_encoded_patches() for root in self.root_partitions])
        print(f"Original image size: {image_width*image_height}, encoded patches: {total_encoded}")
    
    def show_tree_at_depth(self, depth: int):
        """"""
        #Create canvas as blank copy of image
        canvas = np.zeros_like(self.image)

        #Ask nodes to fill it at level N
        for root in self.root_partitions:
            x1, x2, y1, y2 = root.local_coordinates
            canvas_subpartition = canvas[x1:x2, y1:y2]
            root.color_patch_recursive(canvas_subpartition, depth)

        return canvas
    
    def activation_map(self, depth: int):
        """"""
        #Create canvas as blank copy of image
        canvas = np.zeros_like(self.image)

        #Ask nodes to fill it at level N
        for root in self.root_partitions:
            x1, x2, y1, y2 = root.local_coordinates
            canvas_subpartition = canvas[x1:x2, y1:y2]
            root.color_activation_map(canvas_subpartition, depth)

        return canvas
    
    def serialize(self) -> bytearray:
        """Serialize the tree to a byte array
        
        Returns
        ----
            The tree as a byte array
        """
        binary_string = bytearray()

        ##Control/Metadata bytes##
        #Serialize image size
        w,h,_ = self.image.shape
        binary_string.extend(list(w.to_bytes(QuadTree.ControlByteSize.DIMENSION.value, 'big')))
        binary_string.extend(list(h.to_bytes(QuadTree.ControlByteSize.DIMENSION.value, 'big')))
        #Serialize partition size
        partition_w, partition_h = self.root_partition_size
        binary_string.extend(list(partition_w.to_bytes(QuadTree.ControlByteSize.PARTITION_SIZE.value, 'big')))
        binary_string.extend(list(partition_h.to_bytes(QuadTree.ControlByteSize.PARTITION_SIZE.value, 'big')))

        #Append every node to the byte array
        for root in self.root_partitions:
            root.serialize_to(binary_string)
        return binary_string

    @classmethod
    def deserialize(cls, byte_array: bytearray) -> QuadTree:
        """Deserialize the bytearray, create a new QuadTree"""
        quad_tree = QuadTree(None)
        # Control bytes #
        image_w, byte_array = int.from_bytes(byte_array[:QuadTree.ControlByteSize.DIMENSION.value], 'big'), byte_array[QuadTree.ControlByteSize.DIMENSION.value:]
        image_h, byte_array = int.from_bytes(byte_array[:QuadTree.ControlByteSize.DIMENSION.value], 'big'), byte_array[QuadTree.ControlByteSize.DIMENSION.value:]
        partition_w, byte_array = int.from_bytes(byte_array[:QuadTree.ControlByteSize.PARTITION_SIZE.value], 'big'), byte_array[QuadTree.ControlByteSize.PARTITION_SIZE.value:]
        partition_h, byte_array = int.from_bytes(byte_array[:QuadTree.ControlByteSize.PARTITION_SIZE.value], 'big'), byte_array[QuadTree.ControlByteSize.PARTITION_SIZE.value:]

        print(f"Detected image size {(image_w, image_h)}")
        print(f"Detected partition size {(partition_w, partition_h)}")

        while byte_array:
            print(f"{len(byte_array)} bytes left")
            byte_array = None

        #Finally: Now that the nodes are built, they can re-generate the image losslessly
        #quad_tree.image = 
        return quad_tree

class QuadTreeNode:
    image_partition: Mat
    local_coordinates: tuple

    children: list[QuadTreeNode] | None
    color: tuple[int, int, int] | None
    mean_color: tuple[int, int, int] | None

    class NodeType(Enum):
        COLOR_PATCH = 0
        CHILDREN_PATCH = 1

    def __init__(self, image_partition: Mat, local_coordinates: tuple) -> None:
        self.image_partition = image_partition
        self.local_coordinates = local_coordinates
        
        self.children = None
        self.mean_color = None
        self.color = None

        self._build()

    def _build(self) -> None:
        """Build the node recursively, create new nodes if necessary. Automatically called when creating a node on an image"""
        w,h,_ = self.image_partition.shape
        
        r,g,b = self.image_partition[0,0] #tuple()
        reference_pix = (r,g,b)

        #Compare pach's pixel values only if necessary
        if w==1 and h==1:
            self.color = reference_pix
            return
        
        #Non-trival case: Do we stop or do we go further = For that, test uniformity of internal pixels
        mask_result = np.zeros_like(self.image_partition)
        #This creates a boolean mask and apply the mask to the image partition. The 'true' results are given the value 1.
        mask_result[self.image_partition != reference_pix] = 1 

        #At least one pix is not the same, decompose in children
        if np.any(mask_result):
            #Cut 4 equal shapes
            w,h,_ = self.image_partition.shape
            half_w, half_h = int(w/2), int(h/2)

            partitions_coordinates = [
                (0, half_w, 0, half_h),
                (half_w, w, 0, half_h),
                (0, half_w, half_h, h),
                (half_w, w, half_h, h)
            ]

            self.children = []
            for partition_coordinate in partitions_coordinates:
                x1,x2, y1,y2 = partition_coordinate
                image_subpartition = self.image_partition[x1:x2, y1:y2]
                child = QuadTreeNode(image_subpartition, partition_coordinate)
                self.children.append(child)
            #Compute mean-color immediatly in order to not have to re-compute it each time afterwards in case of partial display of the quadtree
            self.mean_color = self.color_or_mean()
        else:
            self.color = reference_pix
    
    def serialize_to(self, byte_array: bytearray) -> None:
        """Append this patch and its children to the given byte array object"""
        if self.children is not None:
            byte_array.append(QuadTreeNode.NodeType.CHILDREN_PATCH.value)
            for child in self.children:
                child.serialize_to(byte_array)
        else:
            byte_array.append(QuadTreeNode.NodeType.COLOR_PATCH.value)
            r,g,b = self.color
            byte_array.extend([r,g,b])

    def color_patch_recursive(self, canvas_partition: Mat, depth: int | None) -> None:
        """Color the given patch. Recursive function with stop condition on depth to alow partial  
        If the patch is uniform, simply color the canvas partition with the given color.
        If the patch is not uniform:
            * If depth is >0, color sub-patches
            * If depth =0, color with mean color of this patch 
        
        """
        if self.color is not None:
            canvas_partition[:,:] = self.color
        else:
            if depth <= 0:
                canvas_partition[:,:] = self.mean_color
            else:
                for child in self.children:
                    x1, x2, y1, y2 = child.local_coordinates
                    canevas_subpartition = canvas_partition[x1:x2, y1:y2]
                    child.color_patch_recursive(canevas_subpartition, depth-1)
    
    def color_activation_map(self, canvas_partition: Mat, depth: int | None) -> None:
        """Color the given patch in red if it is activated at this level"""
        if depth <= 0:
            if self.children is None:
                canvas_partition[:,:] = (220,20,60)
        else:
            if self.children is not None:
                for child in self.children:
                    x1, x2, y1, y2 = child.local_coordinates
                    canevas_subpartition = canvas_partition[x1:x2, y1:y2]
                    child.color_activation_map(canevas_subpartition, depth-1)

    def color_or_mean(self) -> tuple[int, int, int]:
        """Return the color of this patch
        
        Returns
        -----
            Color as RGB if this patch is uniform, mean of the children's colors if the patch has children"""
        if self.color:
            return self.color

        colors = [child.color_or_mean() for child in self.children]
        return np.array(colors).mean(0)

    def count_encoded_patches(self):
        #Trivial case: We are a single patch
        if self.color is not None:
            return 1
        
        return sum([child.count_encoded_patches() for child in self.children])

SyntaxError: invalid syntax (2660890603.py, line 114)

test

In [ ]:
# Change variables here
image_name = "page-SD3-1003.png"

partition_size = (64,64)

In [ ]:
image = cv2.imread(f'./in/{image_name}')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

quadTree = QuadTree(image)

quadTree.build_quad(partition_size)

* Test: See all subpartitions by calling the color method with a depth of 0 (no children called)

In [ ]:
colored_roots = quadTree.show_tree_at_depth(0)

fig, ax = plt.subplots(1, 2)
fig.suptitle(f'Quadtree encoding image {image_name}', fontweight='bold')

ax[0].imshow(image)
ax[0].set_title('Original image')
ax[0].set_axis_off()

ax[1].imshow(colored_roots)
ax[1].set_title('Quadtree image')
ax[1].set_axis_off()

In [ ]:
iterative_images = []
min_dim_size = partition_size[0] #Square dimensions, just take one side
iteration_index = 0
while min_dim_size>=1:
    iterative_images.append(quadTree.show_tree_at_depth(iteration_index))
    min_dim_size /= 2
    iteration_index += 1


fig, ax = plt.subplots(1, len(iterative_images) + 1, figsize=(30,10))
fig.suptitle(f'Quadtree encoding image {image_name}', fontweight='bold')

ax[0].imshow(image)
ax[0].set_title('Original image')
ax[0].set_axis_off()

for i, iter_image in enumerate(iterative_images):
    ax[i+1].imshow(iter_image)
    ax[i+1].set_title(f'Image, quadtree depth of {i}')
    ax[i+1].set_axis_off()

In [ ]:
iterative_images = []
min_dim_size = partition_size[0] #Square dimensions, just take one side
iteration_index = 0
while min_dim_size>=1:
    iterative_images.append(quadTree.show_tree_at_depth(iteration_index))
    min_dim_size /= 2
    iteration_index += 1


fig, ax = plt.subplots(1, len(iterative_images), figsize=(30,10))
fig.suptitle(f'Image substractions over iterations {image_name}', fontweight='bold')

ax[0].imshow(image)
ax[0].set_title('Original image')
ax[0].set_axis_off()

for i in range(len(iterative_images)-1):
    sub_im = cv2.subtract(iterative_images[i+1], iterative_images[i])
    ax[i+1].imshow(sub_im)
    ax[i+1].set_title(f'Image, quadtree depth of {i}')
    ax[i+1].set_axis_off()

In [ ]:
iterative_images = []
min_dim_size = partition_size[0] #Square dimensions, just take one side
iteration_index = 0
while min_dim_size>=1:
    iterative_images.append(quadTree.activation_map(iteration_index))
    min_dim_size /= 2
    iteration_index += 1

1
fig, ax = plt.subplots(1, len(iterative_images)+1, figsize=(30,10))
fig.suptitle(f'Quadtree nodes activation over image {image_name}', fontweight='bold')

ax[0].imshow(image)
ax[0].set_title('Original image')
ax[0].set_axis_off()


iterative_activation_image = np.zeros_like(image)
for i in range(len(iterative_images)):
    #Extract current image, display the addition of both
    current_activation_image = iterative_images[i]
    ax[i+1].imshow(np.add(iterative_activation_image, current_activation_image))
    ax[i+1].set_title(f'Image, quadtree depth of {i}')
    ax[i+1].set_axis_off()
    #Add current activation map to iterative one, as white pixels
    current_activation_image[current_activation_image[:,:] != (0,0,0)] = 255
    iterative_activation_image = np.add(iterative_activation_image, current_activation_image)

In [ ]:
serialized_image = quadTree.serialize()

serialized_size_bytes = len(serialized_image)
serialized_size_kbytes = np.round(serialized_size_bytes/1000, 3)
image_size_kbytes = np.round(image.size/1000, 3)

print(f"Image size: {image_size_kbytes} kb")
print(f"Quadtree stored size: {serialized_size_kbytes}kb")
print(f"Ratio: The size is reduced by a factor or {np.round(image.size/serialized_size_bytes,2)}")
print(f"New storage takes only {100*np.round(serialized_size_bytes/image.size, 2)}% of RGB storage")

Rebuilt image from serialized tree

In [ ]:
rebuilt_quad_tree = QuadTree.deserialize(serialized_image)